在本章中，我们将：
- 定义一个类来处理模型训练。
- 实现构造方法。
- 了解类的公共、受保护和私有方法之间的区别。

# Imports
整个代码中需要的和任何给定章节中使用的所有库都在一开始就导入了。对于本章，我们需要以下导入：

In [1]:
import numpy as np
import datetime

import torch
import torch.optim as optim
import torch.nn as nn
import torch.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

# Going Classy
## 为模型训练构建类 
到目前为止， %%writefile 方法帮助我们将代码组织成三个不同的部分：数据准备、模型配置和模型训练。不过，在本章末尾，重新思考训练循环，我们遇到了它的一些局限性，例如，如果不编辑模型训练代码，就无法选择不同数量的 epoch。
## The class
让我们开始使用一个相当原始的名称来定义我们的类： StepByStep 。因为我们是从头开始的，所以我们要么不指定父类，要么从基本的 object 类继承它。我更喜欢后者，所以我们的类定义如下所示：
```python
# A completely empty (and useless) class
class StepByStep(object):
	pass
```
## 构造函数
“我们从哪里开始建立一个类？”那将是构造函数； __init__(self) 方法，我们在处理模型和数据集类时已经见过几次。

构造函数定义组成类的部分。这些部分是类的属性。典型的属性包括：
-  用户提供的参数。
-  创建时不可用的其他对象的占位符（非常类似于延迟参数）。
-  我们可能想要跟踪的变量。
-  使用一些参数和高阶函数动态构建的函数。

## Arguments 参数 
让我们从参数开始，这是需要由用户指定的部分。在“重新思考训练循环”这一章的开头，我们问自己：“如果我们使用不同的优化器、损失函数甚至模型，训练循环中的代码会发生变化吗？”答案过去是，现在仍然是，不会改变。因此，**优化器、损失和模型**这三个要素将是我们的主要论点。用户需要指定这些；我们无法自己计算出来。

但是还需要一条信息；用于训练模型的设备。我们不会要求用户通知它，而是会自动检查是否有可用的 GPU，如果没有则回退到 CPU。但是我们仍然希望给用户一个使用不同设备的机会，因此，我们添加了一个非常简单的方法，名为 to ，允许用户指定一个设备。添加所有参数后，我们的构造函数 ( __init__ ) 方法最初将如下所示：

In [ ]:
class StepByStep(object):
    #三大参数：模型，损失函数，优化器
    def __init__(self, model, loss_fn, optimizer):
        # Here we define the attributes of our class
        # We start by storing the arguments as attributes 
        # to use them later
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        # Let's send the model to the specified device right away
        self.model.to(self.device)
        
    def to(self, device):
        # This method allows the user to specify a different device
        # It sets the corresponding attribute (to be used later in
        # the mini-batches) and sends the model to the device
        self.device = device
        self.model.to(self.device)        

## Placeholders 占位符 
接下来，让我们处理占位符或延迟参数。我们希望用户最终提供其中一些，因为它们不一定是必需的。在我们的课程中，还有另外三个元素属于此类：训练和验证数据加载器以及与 TensorBoard 交互的摘要编写器。带有附加代码的构造函数如下所示：

In [2]:
class StepByStep(object):
    def __init__(self, model, loss_fn, optimizer):
        # Here we define the attributes of our class
        
        # We start by storing the arguments as attributes 
        # to use them later
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        # Let's send the model to the specified device right away
        self.model.to(self.device)
        
        # These attributes are defined here, but since they are
        # not available at the moment of creation, we keep them None
        self.train_loader = None
        self.val_loader = None
        self.writer = None

换句话说，我们的 StepByStep 类由参数、模型、损失函数和优化器的特定组合定义，然后可用于在任何兼容数据集上执行模型训练。验证数据加载器不是必需的（尽管建议使用），摘要编写器绝对是可选的。因此，该类应该实现允许用户稍后通知这些方法的方法。这两个方法都应该放在 StepByStep 类中和构造方法之后：

In [3]:
def set_loaders(self, train_loader, val_loader=None):
    # This method allows the user to define which train_loader 
    # (and val_loader, optionally) to use
    # Both loaders are then assigned to attributes of the class
    # So they can be referred to later
    self.train_loader = train_loader
    self.val_loader = val_loader

def set_tensorboard(self, name, folder='runs'):
    # This method allows the user to create a SummaryWriter to 
    # interface with TensorBoard
    suffix = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    self.writer = SummaryWriter('{}/{}_{}'.format(
        folder, name, suffix
    ))

“为什么我们需要为 val_loader 指定一个默认值？它的占位符值已经是 None 。”由于验证加载器是可选的，因此在方法定义中为特定参数设置默认值可以使用户在调用方法时不必提供该参数。在我们的例子中，最佳默认值与我们在为验证加载器指定占位符时选择的值相同： None 。
## Variables 变量 
然后，我们可能想要跟踪一些变量。典型的例子是 epoch 的数量以及训练和验证损失。这些变量很可能由类在内部计算和更新。带有附加代码的构造函数将如下所示，类似于我们对占位符所做的：

In [ ]:
class StepByStep(object):
    def __init__(self, model, loss_fn, optimizer):
        # Here we define the attributes of our class
        
        # We start by storing the arguments as attributes 
        # to use them later
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        # Let's send the model to the specified device right away
        self.model.to(self.device)
        
        # These attributes are defined here, but since they are
        # not available at the moment of creation, we keep them None
        self.train_loader = None
        self.val_loader = None
        self.writer = None

        # These attributes are going to be computed internally
        self.losses = []
        self.val_losses = []
        self.total_epochs = 0

“难道我们不能在第一次使用它们时就设置这些变量吗？”

是的，我们可以，而且我们可能会侥幸逃脱，因为我们的课程非常简单。随着类变得越来越复杂，它可能会导致问题。因此，最好的做法是在构造方法中定义一个类的所有属性。

更新后的 StepByStep 类现在如下所示

In [ ]:
class StepByStep(object):
    def __init__(self, model, loss_fn, optimizer):
        # Here we define the attributes of our class
        
        # We start by storing the arguments as attributes 
        # to use them later
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        # Let's send the model to the specified device right away
        self.model.to(self.device)
        
        # These attributes are defined here, but since they are
        # not available at the moment of creation, we keep them None
        self.train_loader = None
        self.val_loader = None
        self.writer = None
        
        # These attributes are going to be computed internally
        self.losses = []
        self.val_losses = []
        self.total_epochs = 0

    def to(self, device):
        # This method allows the user to specify a different device
        # It sets the corresponding attribute (to be used later in
        # the mini-batches) and sends the model to the device
        self.device = device
        self.model.to(self.device)

    def set_loaders(self, train_loader, val_loader=None):
        # This method allows the user to define which train_loader 
        # (and val_loader, optionally) to use
        # Both loaders are then assigned to attributes of the class
        # So they can be referred to later
        self.train_loader = train_loader
        self.val_loader = val_loader

    def set_tensorboard(self, name, folder='runs'):
        # This method allows the user to create a SummaryWriter to 
        # interface with TensorBoard
        suffix = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
        self.writer = SummaryWriter('{}/{}_{}'.format(
            folder, name, suffix
        ))

## Functions 函数
### 创建函数属性
为方便起见，有时创建作为函数的属性很有用，这些属性将在类内的其他地方调用。在我们的例子中，我们可以使用我们在重新思考训练循环一章中定义的高阶函数（分别是辅助函数#1 和#3）来创建 train_step 和 val_step 。它们都以一个模型、一个损失函数和一个优化器作为参数，所有这些都是我们 StepByStep 类在构造时已知的属性。下面的代码将是我们构造函数方法的最终更新版本：

In [ ]:
class StepByStep(object):
    def __init__(self, model, loss_fn, optimizer):
        # Here we define the attributes of our class
        
        # We start by storing the arguments as attributes 
        # to use them later
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        # Let's send the model to the specified device right away
        self.model.to(self.device)
        
        # These attributes are defined here, but since they are
        # not available at the moment of creation, we keep them None
        self.train_loader = None
        self.val_loader = None
        self.writer = None

        # These attributes are going to be computed internally
        self.losses = []
        self.val_losses = []
        self.total_epochs = 0

        # Creates the train_step function for our model, 
        # loss function and optimizer
        # Note: there are NO ARGS there! It makes use of the class
        # attributes directly
        self.train_step = self._make_train_step()
        # Creates the val_step function for our model and loss
        self.val_step = self._make_val_step()

将所有代码拼凑在一起后，您的 StepByStep 类应该如下所示：

In [ ]:
class StepByStep(object):
    def __init__(self, model, loss_fn, optimizer):
        # Here we define the attributes of our class
        # We start by storing the arguments as attributes 
        # to use them later
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        # Let's send the model to the specified device right away
        self.model.to(self.device)
        
        # These attributes are defined here, but since they are
        # not available at the moment of creation, we keep them None
        self.train_loader = None
        self.val_loader = None
        self.writer = None

        # These attributes are going to be computed internally
        self.losses = []
        self.val_losses = []
        self.total_epochs = 0

        # Creates the train_step function for our model, 
        # loss function and optimizer
        # Note: there are NO ARGS there! It makes use of the class
        # attributes directly
        self.train_step = self._make_train_step()
        # Creates the val_step function for our model and loss
        self.val_step = self._make_val_step()

    def to(self, device):
        # This method allows the user to specify a different device
        # It sets the corresponding attribute (to be used later in
        # the mini-batches) and sends the model to the device
        self.device = device
        self.model.to(self.device)        
        
    def set_loaders(self, train_loader, val_loader=None):
        # This method allows the user to define which train_loader 
        # (and val_loader, optionally) to use
        # Both loaders are then assigned to attributes of the class
        # So they can be referred to later
        self.train_loader = train_loader
        self.val_loader = val_loader

    def set_tensorboard(self, name, folder='runs'):
        # This method allows the user to create a SummaryWriter to 
        # interface with TensorBoard
        suffix = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
        self.writer = SummaryWriter('{}/{}_{}'.format(
            folder, name, suffix
        ))